In [1]:
import os

In [2]:
%pwd

'c:\\Users\\saad1\\Used_Car_Prediction_E2E\\research'

In [3]:
cd ..

c:\Users\saad1\Used_Car_Prediction_E2E


In [4]:
%pwd

'c:\\Users\\saad1\\Used_Car_Prediction_E2E'

In [6]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("research/cardekho_cleaned.csv")

In [9]:
df.head()

,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796.0,46.30,5,120000
1,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197.0,82.00,5,550000
2,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197.0,80.00,5,215000
3,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998.0,67.10,5,226000
4,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498.0,98.59,5,570000


In [12]:
X = df.drop('selling_price', axis=1)
y = df['selling_price']

In [16]:
%pip install -q catboost

Note: you may need to restart the kernel to use updated packages.


'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
from catboost import CatBoostRegressor

In [18]:
num_features = X.select_dtypes(exclude="object").columns
onehot_columns = ['seller_type','fuel_type','transmission_type']
binary_columns = ['brand', 'model']

In [19]:
num_features, onehot_columns, binary_columns

(Index(['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats'], dtype='object'),
 ['seller_type', 'fuel_type', 'transmission_type'],
 ['brand', 'model'])

In [20]:
%pip install -q category_encoders

Note: you may need to restart the kernel to use updated packages.


'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.


In [21]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders.binary import BinaryEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
binary_transformer = BinaryEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, onehot_columns),
         ("StandardScaler", numeric_transformer, num_features),
        ("BinaryEncoder", binary_transformer, binary_columns)
        
    ]
)

In [22]:
X = preprocessor.fit_transform(X)

In [23]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((12328, 29), (3083, 29))

In [24]:
cb = CatBoostRegressor(max_depth=8, learning_rate=0.05 ,verbose=False)
model = cb.fit(X_train,y_train)


In [30]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

import numpy as np

In [31]:
def eval_metrics(actual,pred):
    rmse = np.sqrt(mean_squared_error(actual,pred))
    mae = mean_absolute_error(actual,pred)
    r2 = r2_score(actual,pred)
    
    return rmse,mae,r2
    
    

In [32]:
y_pred = model.predict(X_test)

In [33]:
(rmse,mae,r2) = eval_metrics(y_test,y_pred)

In [34]:
print("RMSE ",rmse)
print("MAE ",mae)
print("R2 ",r2)

RMSE  89927.24706666896
MAE  62073.246187130586
R2  0.9445789365805792
